In [10]:
import numpy as np 

  

# P1 = [129.1, 43.1, 1180.1] # RAI

# P2 = [126.2, 225.5, 956.7] # LAI 

P1 = [-129.1, -43.1, -1180.1] # LPS 

P2 = [126.2, -225.5, -956.7] # LPS 

Origin = np.minimum(P1,P2) 

ROIExtent = np.absolute(np.subtract(P1,P2)) 

DesiredSpacing = 0.38 

DesiredSize  = ROIExtent / DesiredSpacing #= round 

DesiredSizeR = np.around(DesiredSize) 

  

print('Size:', DesiredSizeR) 

print('Origin:', Origin) 

print('Spacing:', DesiredSpacing) 

# print(ROIExtent) 

# print(DesiredSize) 

Size: [672. 480. 588.]
Origin: [ -129.1  -225.5 -1180.1]
Spacing: 0.38


In [ ]:
### This Code reads in a defined recentered ZUNU_vida-lung-recentered.nii.gz
### and finds the bounds of the lungs to Upsample the actual image given by the 
## README below
## The Vx3 image that is produced is about 700 MB whereas typical
## Vx3's are around 200-400 MB so the image takes a lot of storage 
## How to make image smaller?
## How to update the min max calculation to be more efficient

import SimpleITK as sitk
import numpy as np
import nibabel as nib

mask_path = "ZUNU_vida-lung-recentered(CASE42).nii.gz"
image_path= "H-17900_IA210257_SPI-IA210257-PBV_EXPIRATION_0.75_Qr40_A_80kV_67503688.nii.gz"

# Read image with Simple ITK
image = sitk.ReadImage(image_path)
mask = sitk.ReadImage(mask_path)

# convert image array to numpy array
transposed_array = sitk.GetArrayFromImage(image)
transposed_mask = sitk.GetArrayFromImage(mask)

#Search Mask array for density values of 0, 20, 30 of lungs
arr1 = transposed_mask

indices_0 = np.argwhere(arr1 == 0)
indices_20 = np.argwhere(arr1 == 20)
indices_30 = np.argwhere(arr1 == 30)

max = indices_20.max(axis=0)
min = indices_20.min(axis=0)
max1 = indices_30.max(axis=0)
min1 = indices_30.min(axis=0)

print("Max20 is ", max)
print("Max30 is ", max1)
print("Min20 is ", min)
print("Min30 is ", min1)

# For Case 42 I find these maxes/minumums
# Max20 is  [521 431 499]
# Max30 is  [506 439 276]
# Min20 is  [ 55  71 249]
# Min30 is  [34 77 40]

# The points below this line (min and max) are calculated using lines 50-95
# Lines 50-95 can definitely be simplified 
# min = (34, 71, 40) #transposed point 
# max = (521, 439, 499) #transposed point

#Lines 47-95 is a bad way to set 
maxxc = max[2]
maxxc1 = max1[2]
# print("Maxxc = ", maxxc)
# print("Maxxc1 = ", maxxc1)
if maxxc > maxxc1:
    maxx = maxxc
if maxxc < maxxc1:
    maxx = maxxc1
minxc = min[2]
minxc1 = min1[2]
if minxc < minxc1:
    minx = minxc
if minxc > minxc1:
    minx = minxc1
# print("Minxc = ", minxc)
# print("Minxc1 = ", minxc1)
maxyc = max[1]
maxyc1 = max1[1]
# print("Maxyc = ", maxyc)
# print("Maxyc1 = ", maxyc1)
if maxyc > maxyc1:
    maxy = maxyc
if maxyc < maxyc1:
    maxy = maxyc1
minyc = min[1]
minyc1 = min1[1]
if minyc < minyc1:
    miny = minyc
if minyc > minyc1:
    miny = minyc1
# print("Minyc = ", minyc)
# print("Minyc1 = ", minyc1)
maxzc = max[0]
maxzc1 = max1[0]
# print("Maxzc = ", maxzc)
# print("Maxzc1 = ", maxzc1)
if maxzc > maxzc1:
    maxz = maxzc
if maxzc < maxzc1:
    maxz = maxzc1
minzc = min[0]
minzc1 = min1[0]
if minzc < minzc1:
    minz = minzc
if minzc > minzc1:
    minz = minzc1
# print("Minzc = ", minzc)
# print("Minzc1 = ", minzc1)

def Physical_Point(x, y, z):
    index = [x, y, z]
    p1 = image.TransformIndexToPhysicalPoint((x, y, z))
    point1 = (p1[0], p1[1], p1[2])
    return(point1)
# x = 40
# y = 71
# z = 34

# Values in sitk.resampleimagefilter need to be Integers not Floats
x = int(minx)
y = int(miny)
z = int(minz)
# Find physical point of minimum
point1 = Physical_Point(x, y, z)

def Physical_Point(i, j, k):
    index = [i, j, k]
    p2 = image.TransformIndexToPhysicalPoint((i, j, k))
    point2 = (p2[0], p2[1], p2[2])
    return(point2)

# i = 499
# j = 439
# k = 521

i = int(maxx)
j = int(maxy)
k = int(maxz)
# Find physical point of max
point2 = Physical_Point(i, j, k)
# Print Physical Points to see them
print("Point 1:", point1)
print("Point 2:", point2)
 
# P1 = [134.3, 61.9, 140.7] # RAI
# P2 = [134.2, 238.6, 57.3] # LAI
# P1 = [-134.3, -61.9, -140.7] # LPS 
# P2 = [134.2, -238.6, 57.3] # LPS 
# Code to make Original Vx3
output_origin = np.minimum(point1,point2) 
ROIExtent = np.absolute(np.subtract(point1,point2)) 
DesiredSpacing = 0.38 

x1 = ROIExtent[0] / DesiredSpacing  
y1 = ROIExtent[1] / DesiredSpacing
z1 = ROIExtent[2] / DesiredSpacing
# Turn Floats into Integers
x2 = int(x1) + 10
y2 = int(y1) + 10
z2 = int(z1) + 10
# Chat GPT gave this to me in a function form but this could be edited 

def resample_image(input_image, new_size, output_origin, output_spacing, output_pixel_type, interpolator):
        # Create a resampler object
        resampler = sitk.ResampleImageFilter()

        #Set the interpolator
        resampler.SetInterpolator(interpolator)

        # Set the reference image for output size, origin, and spacing
        # resampler.SetOutputParametersFromImage(input_image)

        # Set the new output size, origin, and spacing
        resampler.SetSize(new_size)
        resampler.SetOutputOrigin(output_origin)
        resampler.SetOutputSpacing(output_spacing)

        # Set the output pixel type
        resampler.SetOutputPixelType(output_pixel_type)

        # Perform the resampling
        output_image = resampler.Execute(input_image)

        return(output_image)
# Setting Variables before calling function        
output_pixel_type = sitk.sitkInt16
output_spacing = (.38, .38, .38)
interpolator = sitk.sitkBSpline3
output_size = (x2, y2, z2)

output_image_path = "../Vx3/Vx3.nrrd"
output_image = resample_image(image, output_size, output_origin, output_spacing, output_pixel_type, interpolator)
sitk.WriteImage(output_image, output_image_path)

print("Vx3 was created!")
print("Output Image Dimensions: ", output_size)
print("Output Image Spacing: ", output_spacing)
print("Output Image Origin: ", output_origin)
print("Output Image Size: ", output_size)